In [1]:
import string
import requests
import json
import random
import time
import pandas as pd
#import api_key

possibleSubjectAreas = ['AGRI','ARTS','BIOC','BUSI','CENG','CHEM','COMP','DECI','DENT',
                'EART','ECON','ENER','ENGI','ENVI','HEAL','IMMU','MATE','MATH','MEDI','NEUR',
                'NURS','PHAR','PHYS','PSYC','SOCI','VETE','MULT']

years = range(2018,1990,-1)


SCOPUS_PAGE_SIZE=25
SCOPUS_MAX_RESULTS_PER_QUERY=5000

api_token = 'your_api_token'
my_api_key = '0b1939facfd1a6d769ef6f31a9b9d92f'#api_key.my_api_key #mine
#default_api_key = '7f59af901d2d86f78a1fd60c1bf9426a' #default


api_url_base = 'https://api.elsevier.com/content/search/scopus'
headers = {'Accept' : 'application/json',}

These functions get a set of authors ID randomly. It then samples papers that are from authors ID in the set. 

getMask gets a random 11 digits integer (scopus author IDs are 11 digits). then we mask some digits with '?' (meaning it's ok whatever their value is). If we mask one digit, it means we fix 10 out of 11 digits (we identify 10 authors). for each more digits we mask, it is a 10x increase. for n masked digits (n is the "masklevel"), we restrict the query to 10^n authors. 

or a given year, subjectArea, authorIDMask (that is, set of authorIDs),  getScopusProcsData then returns papers from authors in that set

In [8]:
def getMask(maskLevel):
    assert (maskLevel<=10) and (maskLevel>0)
    authorIDMask = list(str(random.randint(1, 99999999999)))
    #pad with leading zeros if smaller than 11 digits
    pad = 11 - len(authorIDMask)
    pad_array = ['0']*pad
    pad_array.extend(authorIDMask)
    authorIDMask = pad_array
    #always mask the first (it's newer and it can bias towards new authors)
    authorIDMask[0]='?'
    mask = random.sample(range(1, 11), maskLevel-1)
    #mask.append(0)
    for i in mask:
        authorIDMask[i]='?'
    completed_authorIDMask="".join(authorIDMask)
    return completed_authorIDMask

#create a fixed mask for titles, consisting of N characters with M ? chars in between
def get_title_mask(num_random_chars, in_between_chars):
    
    s = '*'
    for i in range(num_random_chars):
        s += '?' * in_between_chars
        c = random.choice(string.ascii_lowercase)
        s+=c
    s += '*'
    return s

#retrieve up to retrieveCount papers for a given year, subjectArea, authorIDMask
# def getScopusProcsData_per_mask(year, subjectArea, maskLevel, retrieveCountForEachMask, api_key):
    
#     results = []
#     noMoreResults = False
#     start=0

#     while (len(results) < retrieveCountForEachMask ) and not noMoreResults:
#         authorIDMask = getMask(maskLevel)
#         query_count = SCOPUS_PAGE_SIZE
#         query = 'query=title-abs-key(a*)%20AND%20AU-ID(' + authorIDMask + ')%20AND%20(srctype(p)%20OR%20srctype(k))%20AND%20(doctype(cp))'
        
#         filter = '&date=' + str(year) + '&subj=' + subjectArea + '&count=' + str(query_count)+ '&start=' + str(start)
#         api_url = api_url_base + '?' + query + filter + '&apiKey=' + api_key + '&httpAccept=application%2Fjson'
#         print(api_url)
#         response = requests.get(api_url, headers=headers)
#         jresponse=[]
        
#         if response.status_code == 200:
#             print('start:' + str(start))
#             r = json.loads(response.content.decode('utf-8'))['search-results']
#             if 'entry' in r:
#                 jresponse = r['entry']
#                 if ('error' not in jresponse[0]):
#                     results.extend( jresponse)
#                     print( 'in getScopusProcsData: retrieved ',len(results),'out of ',retrieveCountForEachMask, s )
#         else: 
#             print('\n \n ******', response.status_code, response.headers, response.text)
            
#         if (len(jresponse)<query_count):
#             break
#         else:
#             start+=query_count

#     return results[:retrieveCountForEachMask]

This functions gets proceedings papers by first starting with a narrow authors mask (10^4 authors) and then increasing the mask if there are not enough papers to fill the requested retrieveCount.

Notice that for EACH scopus query we get a different author mask, to maximise randomness

In [9]:
# def getDOIsRandomly(year,subjectArea, maskLevel=4, 
#                     retrieveCount=1000,  sleepInterval=10, api_key='None'):
        
        
#     retrieveCountForEachMask=retrieveCount
#     results = []
    
    
#     while (len(results)<retrieveCount) and (maskLevel<=10):
#         start=0
#         results_to_be_retrieved = min(retrieveCountForEachMask, retrieveCount - len(results))
#         if (results_to_be_retrieved <=0): 
#             break
                    
#         callResults =  getScopusProcsData_per_mask(year, subjectArea, maskLevel, results_to_be_retrieved, api_key)
#         results.extend(callResults)

#         if (len(callResults) < results_to_be_retrieved):
#             maskLevel += 1
#             print ('increasing mask level to: ', maskLevel,'after results so far:',len(results),s)
#         else:
#             start+=SCOPUS_PAGE_SIZE

#     return results



# def getDOIsRandomly_noyearsubj(maskLevel=4, maxMaskLevel=7, retrieveCount=1000,   api_key='None'):
        
#     retrieveCountForEachMask=retrieveCount
#     results = []
#     attempts_with_this_mask_level=0
#     MAX_ATTEMPTS_AT_MASK = 10
    
#     while (len(results)<retrieveCount) and (maskLevel<=maxMaskLevel):
#         print('(start) maskLevel now at ', maskLevel)
#         authorIDMask = getMask(maskLevel)
#         titleMask = '*a*'
#         print ('authorIDMask',authorIDMask)
#         query_count = 10#SCOPUS_PAGE_SIZE
 

# #        query = 'query=title-abs-key(a*)%20AND%20AU-ID(' + authorIDMask + ')%20AND%20(srctype(p)%20OR%20srctype(k))%20AND%20(doctype(cp))'
#         query = 'query=title('+titleMask+')%20AND%20(srctype(p)%20OR%20srctype(k))%20AND%20(doctype(cp))'

#         filter = '&count=' + str(query_count)#+ '&start=' + str(start)
#         api_url = api_url_base + '?' + query + filter + '&apiKey=' + api_key + '&httpAccept=application%2Fjson'
#         print (api_url)
#         response = requests.get(api_url, headers=headers)
#         jresponse=[]
        
#         if response.status_code == 200:
#             r = json.loads(response.content.decode('utf-8'))['search-results']
#             if 'entry' in r:
#                 jresponse = r['entry']
#                 if ('error' not in jresponse[0]):
#                     results.extend(jresponse)
#                 else:
#                     jresponse=[]
#         else: 
#             print('\n \n ******', response.status_code, response.headers, response.text)
                    
#         print('mask level', maskLevel, 'jresponse:', len(jresponse), 
#               'results: ', len(results), 'attempt:', attempts_with_this_mask_level,
#               'must increase:', attempts_with_this_mask_level == MAX_ATTEMPTS_AT_MASK-1 )

    
#         if (len(jresponse)==0):
#             if (attempts_with_this_mask_level == MAX_ATTEMPTS_AT_MASK-1):
#                 maskLevel+=1
#                 print('increasing maskLevel, now at ', maskLevel)
#                 attempts_with_this_mask_level=0
#             else:
#                 attempts_with_this_mask_level+=1
#         else:
#             attempts_with_this_mask_level=0 # reset, if we get results
#         print('(end) maskLevel now at ', maskLevel)


#     return results





In [10]:
%%capture capt



UsageError: %%capture is a cell magic, but the cell body is empty.


# sample scopus based on title mask - by subject and year

In [11]:
# def retrieve_randomly(retrieveCount=1000):
#     #for i in range(10):
#     frames=[]
#     papers = getDOIsRandomly_titlemask(maskLevel=7, maxMaskLevel=9, retrieveCount=retrieveCount, api_key = my_api_key )
#     df = pd.DataFrame(papers)
#     print ('results length: ', df.shape[0])
# #     if 'prism:doi' not in list(df.columns):
# #         #print('no doi for '+ f)
# #         continue
#     to_drop = [c for c in list(df.columns) if c not in to_keep]
#     df.drop(columns=to_drop, inplace=True)
#     df.dropna(subset=['prism:doi'],inplace=True)
#     df.set_index('prism:doi', inplace=True, drop=False)
#     df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
#     df.drop_duplicates(subset='prism:doi', inplace=True)
# #     df.drop_duplicates(subset='prism:doi', inplace=True)  
# #     frames.append(df)


#     return df


def run_single_query(start,query_count,year,subjectArea,api_key,titleMask,api_url_base):
    query = 'query=title('+titleMask+')%20AND%20(srctype(p)%20OR%20srctype(k))%20AND%20(doctype(cp))'
    filter = '&count=' + str(query_count)+ '&start=' + str(start)+ '&date=' + str(year) + '&subj=' + subjectArea 
    api_url = api_url_base + '?' + query + filter + '&apiKey=' + api_key + '&httpAccept=application%2Fjson'
    #print(api_url)
    response = requests.get(api_url, headers=headers)
    jresponse=[]
    if response.status_code == 200:
        r = json.loads(response.content.decode('utf-8'))['search-results']
        if 'entry' in r:
            jresponse = r['entry']
            if ('error' not in jresponse[0]):
                return jresponse
    return []
#             else:
#                 return []
#         else: 
#             print('\n \n ******', response.status_code, response.headers, response.text)
                    
        #print('jresponse:', len(jresponse), 'results: ', len(results))
    


def getDOIsRandomly_titlemask(retrieveCount=1000,   api_key='None', to_keep=[]):
    #start=0
    results = []
    query_count = 25#SCOPUS_PAGE_SIZE
    frames=[]
    
    while (len(results)<retrieveCount) :   
        #pick a random subject
        subjectID = random.randint(0, len(possibleSubjectAreas)-1)
        subjectArea = possibleSubjectAreas[subjectID]
        print ('subject:', subjectArea)    
        #pick a random year
        year = random.randint(2000, 2019)
        print ('year:', year)
        #pick a random title mask
        titleMask = get_title_mask(2,4)
    
    # query scopus with these parameters
    # also: query starts from count 4970. if this has no results, then this means this title is sufficiently random
    # not to get into sorting bias, and we then query from the start


        probe = run_single_query(4975,query_count,year,subjectArea,api_key,titleMask,api_url_base)
        if (len(probe)==0): # we can start from the beginning, it is an unbiased sample
#             continue #go to next iteration
#         else:
            start=0
            while (len(results)<retrieveCount) and (start<4970) :
                single_query_results = run_single_query(start,query_count,year,subjectArea,api_key,titleMask,api_url_base)
                if (len(single_query_results) >0):
                    start+=SCOPUS_PAGE_SIZE
                    results.extend(single_query_results)
                else:
                    break
                
# #        query = 'query=title-abs-key(a*)%20AND%20AU-ID(' + authorIDMask + ')%20AND%20(srctype(p)%20OR%20srctype(k))%20AND%20(doctype(cp))'
#         query = 'query=title('+titleMask+')%20AND%20(srctype(p)%20OR%20srctype(k))%20AND%20(doctype(cp))'
#         filter = '&count=' + str(query_count)+ '&start=' + str(start)+ '&date=' + str(year) + '&subj=' + subjectArea 
#         api_url = api_url_base + '?' + query + filter + '&apiKey=' + api_key + '&httpAccept=application%2Fjson'
#         print (api_url)
#         response = requests.get(api_url, headers=headers)
#         jresponse=[]
        
#         if response.status_code == 200:
#             r = json.loads(response.content.decode('utf-8'))['search-results']
#             if 'entry' in r:
#                 jresponse = r['entry']
#                 if ('error' not in jresponse[0]):
#                     results.extend(jresponse)
#                 else:
#                     jresponse=[]
#         else: 
#             print('\n \n ******', response.status_code, response.headers, response.text)
                    
            print('results: ', len(results))

#         if (len(jresponse) >0):
#             start+=SCOPUS_PAGE_SIZE
#         else:
#             break

            df = pd.DataFrame(results)
            df['subj'] = subjectArea
            df['year'] = year
            if 'prism:doi'  in list(df.columns):
                #print('no doi for '+ f)
                df.dropna(subset=['prism:doi'],inplace=True)
                df.set_index('prism:doi', inplace=True, drop=False)
                df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
                frames.append(df)
                to_drop = [c for c in list(df.columns) if c not in to_keep]
                df.drop(columns=to_drop, inplace=True)
    df = pd.concat(frames)
    df.drop_duplicates(subset='prism:doi', inplace=True)
    return df



# def retrieve_randomly(retrieveCount=1000):
#     #for i in range(10):
# #     frames=[]
# #     for i in range (10):
#     df = getDOIsRandomly_titlemask(retrieveCount=retrieveCount, api_key = my_api_key )
# #         print ('results length: ', df.shape[0])
# #         if 'prism:doi' not in list(df.columns):
# #             #print('no doi for '+ f)
# #             continue
# #         df.dropna(subset=['prism:doi'],inplace=True)
# #         df.set_index('prism:doi', inplace=True, drop=False)
# #         df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
# #         frames.append(df)
        
# #     df=pd.concat(frames)
#     df.drop_duplicates(subset='prism:doi', inplace=True)


#     return df






# def retrieve_by_subject_and_year(years, subjects, retrieveCount,folderName):
#     for y in [2011,2012]:
#         frames=[]
#         for s in possibleSubjectAreas:
#             papers = getDOIsRandomly(year=y, subjectArea=s,  maskLevel=6, 
#                 retrieveCount=RETRIEVE_COUNT, sleepInterval=10, api_key = my_api_key )

#             df = pd.DataFrame(papers)
#             print ('results length: ',s, df.shape[0])


#             if 'prism:doi' not in list(df.columns):
#                 #print('no doi for '+ f)
#                 continue
#             to_drop = [c for c in list(df.columns) if c not in to_keep]
#             df.drop(columns=to_drop, inplace=True)
#             df['subj'] = s
#             df['year'] = y
#             df.dropna(subset=['prism:doi'],inplace=True)
#             df.set_index('prism:doi', inplace=True, drop=False)
#             df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
#             df.drop_duplicates(subset='prism:doi', inplace=True)
#             frames.append(df)


#         df=pd.concat(frames)
#         df.drop_duplicates(subset='prism:doi', inplace=True)
#         papers_filename = folderName+'/'+str(y)+'-paper_results.csv'
#         df.to_csv(papers_filename)
#         #return df




In [12]:


#https://api.elsevier.com/content/search/scopus?start=0&date=2015&count=25&subj=ECON&query=title(o*)+AND+srctype(p)&apiKey=7f59af901d2d86f78a1fd60c1bf9426a&httpAccept=application%2Fjson

# results = []


# to_keep=['prism:doi','citedby-count','openaccess','affiliation',
#  'openaccessFlag','prism:coverDate','dc:title',
#  'prism:coverDisplayDate', 'prism:pageRange',
#  'prism:publicationName','subtype','subj', 'year', 'country']

# RETRIEVE_COUNT=5000

# dfbsy =retrieve_by_subject_and_year([2011,2012], possibleSubjectAreas, RETRIEVE_COUNT, 'randomdata')    

        


In [13]:
import time
ts = time.time()
print(int(ts))

results = []


to_keep=['prism:doi','citedby-count','openaccess','affiliation',
 'openaccessFlag','prism:coverDate','dc:title',
 'prism:coverDisplayDate', 'prism:pageRange',
 'prism:publicationName','subtype','subj', 'year', 'country']

RETRIEVE_COUNT=100


    #     df=pd.concat(frames)
df = getDOIsRandomly_titlemask(RETRIEVE_COUNT,api_key=my_api_key,to_keep=to_keep)
df['citedby-count']=pd.to_numeric(df['citedby-count'])
papers_filename = 'data/scopus_data/title_mask/rand-paper_results'+str(int(time.time()))+'.csv'
df.to_csv(papers_filename)




1542800126
subject: DENT
year: 2006
results:  0
subject: MATE
year: 2013
results:  25


TypeError: drop() got an unexpected keyword argument 'columns'

In [ ]:
df['prism:doi'].is_unique

In [ ]:
df['citedby-count'].describe()

In [ ]:
# df['citedby-count']=pd.to_numeric(df['citedby-count'])

In [ ]:
df.columns

In [ ]:
ddf['citedby-count'].unique()

In [ ]:
df.columns

In [ ]:
df.shape

In [ ]:
# import os
# def load_clean_and_merge(folder):
#     files = os.listdir(folder+'/')
#     files = [f for f in files if 'csv' in f]
#     frames=[]
#     for f in files:
#         df = pd.read_csv(folder+'/'+f)
#         df['citedby-count']=pd.to_numeric(df['citedby-count'])
#         print(df.shape)
#         frames.append(df) 
        
#     df = pd.concat(frames)
#     return df
        

In [ ]:
# cdf = load_clean_and_merge('randomdata')

In [ ]:
# cdf.shape
# cdf.columns

In [ ]:
# cdf['citedby-count'].describe()

In [ ]:
cdf.groupby(['year', 'subj'])['citedby-count'].describe()

In [ ]:
cdf.groupby('subj').size()
